<a href="https://colab.research.google.com/github/nsriniva/DS-Unit-2-Applied-Modeling/blob/master/module2-wrangle-ml-datasets/LS_DS_232_assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Lambda School Data Science

*Unit 2, Sprint 3, Module 2*

---


# Wrangle ML datasets

- [ ] Continue to clean and explore your data. 
- [ ] For the evaluation metric you chose, what score would you get just by guessing?
- [ ] Can you make a fast, first model that beats guessing?

**We recommend that you use your portfolio project dataset for all assignments this sprint.**

**But if you aren't ready yet, or you want more practice, then use the New York City property sales dataset for today's assignment.** Follow the instructions below, to just keep a subset for the Tribeca neighborhood, and remove outliers or dirty data. [Here's a video walkthrough](https://youtu.be/pPWFw8UtBVg?t=584) you can refer to if you get stuck or want hints!

- Data Source: [NYC OpenData: NYC Citywide Rolling Calendar Sales](https://data.cityofnewyork.us/dataset/NYC-Citywide-Rolling-Calendar-Sales/usep-8jbt)
- Glossary: [NYC Department of Finance: Rolling Sales Data](https://www1.nyc.gov/site/finance/taxes/property-rolling-sales-data.page)

In [1]:
from collections import OrderedDict
from math import isclose
import zipfile 
from urllib.request import urlopen
import io

import requests
from bs4 import BeautifulSoup

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from scipy import stats
from scipy.stats import chi2_contingency

from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from sklearn.linear_model import LinearRegression

from sklearn.model_selection import train_test_split

# For details about the data cleanup, please see 
# https://github.com/nsriniva/DS-Unit-2-Applied-Modeling/blob/master/CleanupOnlineNewsPopularity.ipynb
# and 'The Dataset' section of
# https://nsriniva.github.io/2020-10-23-DSPT9-Unit1-BuildProject/

# Cleaned up and uploaded csv data file from 
# https://archive.ics.uci.edu/ml/machine-learning-databases/00332/OnlineNewsPopularity.zip 
# in
# https://archive.ics.uci.edu/ml/datasets/Online+News+Popularity
# to my github repo as
# https://github.com/nsriniva/DS-Unit-2-Applied-Modeling/blob/master/OnlineNewsPopularity.csv.zip?raw=true

# The associated names file is available at
# https://raw.githubusercontent.com/nsriniva/DS-Unit-2-Applied-Modeling/master/OnlineNewsPopularity.names


onp_url = 'https://github.com/nsriniva/DS-Unit-2-Applied-Modeling/blob/master/OnlineNewsPopularity.csv.zip?raw=true'

onp_df = pd.read_csv(onp_url, compression='zip')

null_values = onp_df.isna().sum().sum()

print(f"There are {['','no'][int(null_values==0)]} invalid values in the dataset!")

# The zscore() method from the scipy.stats package is used to compute z scores 
# for the shares values. These z scores is compared against the specified  
# sigma value to generate a boolean filter array that could be used to 
# paritition the dataset based on whether the zscore is greater than the
# specified sigma.
def get_sigma_filter(df, sigma=0.5):
  z = np.abs(stats.zscore(df.shares))
  return np.where(z>sigma)[0]

# Use the boolean filter array provided by get_sigma_filter() to
# ignore entries with zscore greater than 0.5 and compute the
# median and max 'shares' values for the remaining entries.
def classification_marks(df):

  shares_info = df.drop(get_sigma_filter(df)).shares

  max = shares_info.max()
  median = shares_info.median()

  return median, max


shares_median = onp_df.shares.median()

print(shares_median)
# Use the medium(median) value to classify articles into 
# unpopular(0) and popular(1)  
onp_df['popularity'] = onp_df.shares.apply(lambda x: 0 if x < shares_median else 1)




There are no invalid values in the dataset!
1400.0


In [2]:
null_values = onp_df.isna().sum().sum()

print(f"There are {['','no'][int(null_values==0)]} invalid values in the dataset!")

display(onp_df.head())

onp_df.data_channel.value_counts()

There are no invalid values in the dataset!


,url,timedelta,n_tokens_title,n_tokens_content,n_unique_tokens,n_non_stop_words,n_non_stop_unique_tokens,num_hrefs,num_self_hrefs,num_imgs,num_videos,average_token_length,num_keywords,kw_min_min,kw_max_min,kw_avg_min,kw_min_max,kw_max_max,kw_avg_max,kw_min_avg,kw_max_avg,kw_avg_avg,self_reference_min_shares,self_reference_max_shares,self_reference_avg_sharess,is_weekend,LDA_00,LDA_01,LDA_02,LDA_03,LDA_04,global_subjectivity,global_sentiment_polarity,global_rate_positive_words,global_rate_negative_words,rate_positive_words,rate_negative_words,avg_positive_polarity,min_positive_polarity,max_positive_polarity,avg_negative_polarity,min_negative_polarity,max_negative_polarity,title_subjectivity,title_sentiment_polarity,abs_title_subjectivity,abs_title_sentiment_polarity,shares,data_channel,weekday,popularity
0,http://mashable.com/2013/01/07/amazon-instant-...,731.0,12.0,219.0,0.663594,1.0,0.815385,4.0,2.0,1.0,0.0,4.680365,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,496.0,496.0,496.000000,0.0,0.500331,0.378279,0.040005,0.041263,0.040123,0.521617,0.092562,0.045662,0.013699,0.769231,0.230769,0.378636,0.100000,0.7,-0.350000,-0.600,-0.200000,0.500000,-0.187500,0.000000,0.187500,593,entertainment,monday,0
1,http://mashable.com/2013/01/07/ap-samsung-spon...,731.0,9.0,255.0,0.604743,1.0,0.791946,3.0,1.0,1.0,0.0,4.913725,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.799756,0.050047,0.050096,0.050101,0.050001,0.341246,0.148948,0.043137,0.015686,0.733333,0.266667,0.286915,0.033333,0.7,-0.118750,-0.125,-0.100000,0.000000,0.000000,0.500000,0.000000,711,bus,monday,0
2,http://mashable.com/2013/01/07/apple-40-billio...,731.0,9.0,211.0,0.575130,1.0,0.663866,3.0,1.0,1.0,0.0,4.393365,6.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,918.0,918.0,918.000000,0.0,0.217792,0.033334,0.033351,0.033334,0.682188,0.702222,0.323333,0.056872,0.009479,0.857143,0.142857,0.495833,0.100000,1.0,-0.466667,-0.800,-0.133333,0.000000,0.000000,0.500000,0.000000,1500,bus,monday,1
3,http://mashable.com/2013/01/07/astronaut-notre...,731.0,9.0,531.0,0.503788,1.0,0.665635,9.0,0.0,1.0,0.0,4.404896,7.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.028573,0.419300,0.494651,0.028905,0.028572,0.429850,0.100705,0.041431,0.020716,0.666667,0.333333,0.385965,0.136364,0.8,-0.369697,-0.600,-0.166667,0.000000,0.000000,0.500000,0.000000,1200,entertainment,monday,0
4,http://mashable.com/2013/01/07/att-u-verse-apps/,731.0,13.0,1072.0,0.415646,1.0,0.540890,19.0,19.0,20.0,0.0,4.682836,7.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,545.0,16000.0,3151.157895,0.0,0.028633,0.028794,0.028575,0.028572,0.885427,0.513502,0.281003,0.074627,0.012127,0.860215,0.139785,0.411127,0.033333,1.0,-0.220192,-0.500,-0.050000,0.454545,0.136364,0.045455,0.136364,505,tech,monday,0


world            9513
entertainment    8384
tech             8219
bus              6980
socmed           2323
lifestyle        2099
culture          1917
u.s.              138
social-good        46
misc               25
Name: data_channel, dtype: int64

In [3]:
from pandas_profiling import ProfileReport
profile = ProfileReport(onp_df, minimal=True).to_notebook_iframe()

profile

In [4]:
# Remove outliers
def remove_outliers(df, sigma=0.5):
  df = df.copy()
  return df.drop(get_sigma_filter(df, sigma))

onp_no_df = remove_outliers(onp_df, 0.25)

In [5]:
target = 'popularity'

# The baseline accuracy or the value we'd get by just guessing that that the
# value is always the majority class
baseline_accuracy = onp_no_df[target].value_counts(normalize=True).max()

print(f'baseline_accuracy = {baseline_accuracy:0.4f}')

baseline_accuracy = 0.5031


In [6]:
# Drop the 'shares' column used to derive 'popularity' along
# with the non predictive 'url' and 'timedelta' columns.
drop_cols = ['shares', 'url', 'timedelta']

# Will use a random split of 64% Training, 16% Validation and 20% Test 

X = onp_no_df.drop(columns=drop_cols).drop(columns=target)
y = onp_no_df[target]

X_train_val, X_test, y_train_val, y_test = train_test_split(X,y,train_size=0.8, random_state=42)

X_train, X_val, y_train, y_val = train_test_split(X_train_val, y_train_val, train_size=0.8, random_state=42)

X_train.shape, X_val.shape, X_test.shape, y_train.shape, y_val.shape, y_test.shape

((22267, 47), (5567, 47), (6959, 47), (22267,), (5567,), (6959,))

In [7]:
display(X_train.head())

,n_tokens_title,n_tokens_content,n_unique_tokens,n_non_stop_words,n_non_stop_unique_tokens,num_hrefs,num_self_hrefs,num_imgs,num_videos,average_token_length,num_keywords,kw_min_min,kw_max_min,kw_avg_min,kw_min_max,kw_max_max,kw_avg_max,kw_min_avg,kw_max_avg,kw_avg_avg,self_reference_min_shares,self_reference_max_shares,self_reference_avg_sharess,is_weekend,LDA_00,LDA_01,LDA_02,LDA_03,LDA_04,global_subjectivity,global_sentiment_polarity,global_rate_positive_words,global_rate_negative_words,rate_positive_words,rate_negative_words,avg_positive_polarity,min_positive_polarity,max_positive_polarity,avg_negative_polarity,min_negative_polarity,max_negative_polarity,title_subjectivity,title_sentiment_polarity,abs_title_subjectivity,abs_title_sentiment_polarity,data_channel,weekday
9904,9.0,302.0,0.534965,1.0,0.656805,33.0,16.0,1.0,0.0,4.685430,7.0,4.0,673.0,254.714286,4600.0,843300.0,281957.142857,2133.909091,7890.798883,4721.160619,983.0,983.0,983.000000,1.0,0.028572,0.028582,0.028572,0.743052,0.171222,0.541250,0.181250,0.043046,0.016556,0.722222,0.277778,0.476923,0.200000,0.8,-0.415000,-0.625,-0.050,0.000000,0.000000,0.500000,0.000000,culture,sunday
186,10.0,498.0,0.522822,1.0,0.651466,9.0,0.0,1.0,0.0,4.676707,9.0,217.0,1200.0,577.000000,0.0,28000.0,5433.333333,0.000000,2240.536313,1022.631548,0.0,0.0,0.000000,0.0,0.359030,0.022229,0.574294,0.022223,0.022224,0.445971,0.199226,0.054217,0.010040,0.843750,0.156250,0.399669,0.050000,0.8,-0.253333,-0.600,-0.100,0.000000,0.000000,0.500000,0.000000,bus,thursday
6228,11.0,625.0,0.482315,1.0,0.674157,6.0,4.0,1.0,0.0,4.478400,3.0,4.0,302.0,174.333333,42700.0,690400.0,268300.000000,3225.681818,3409.183231,3294.193735,1700.0,3800.0,2625.000000,0.0,0.066667,0.066670,0.066669,0.066667,0.733327,0.543358,0.202493,0.056000,0.012800,0.813953,0.186047,0.416685,0.100000,1.0,-0.279613,-0.500,-0.100,0.483333,0.266667,0.016667,0.266667,tech,wednesday
17920,10.0,312.0,0.608974,1.0,0.798883,9.0,5.0,1.0,0.0,4.871795,10.0,-1.0,1100.0,333.666667,0.0,843300.0,170725.400000,0.000000,3760.714286,2274.793517,4300.0,22300.0,16300.000000,1.0,0.020807,0.020035,0.816878,0.020072,0.122208,0.562302,-0.105231,0.019231,0.038462,0.333333,0.666667,0.386616,0.136364,0.5,-0.485417,-1.000,-0.125,0.000000,0.000000,0.500000,0.000000,world,sunday
13053,12.0,534.0,0.504950,1.0,0.631579,30.0,3.0,26.0,1.0,4.722846,8.0,4.0,594.0,173.714286,0.0,843300.0,279800.000000,0.000000,4985.294118,3452.600435,532.0,18000.0,6510.666667,0.0,0.025000,0.785307,0.025000,0.139693,0.025000,0.644636,0.163985,0.029963,0.020599,0.592593,0.407407,0.540625,0.200000,1.0,-0.404040,-0.800,-0.100,0.000000,0.000000,0.500000,0.000000,entertainment,monday


In [8]:
display(y_train.value_counts(normalize=True))
baseline_accuracy = y_train.value_counts(normalize=True).max()

print(f'baseline_accuracy = {baseline_accuracy:0.4f}')

0    0.500472
1    0.499528
Name: popularity, dtype: float64

baseline_accuracy = 0.5005


In [9]:
from category_encoders import OrdinalEncoder, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.tree import DecisionTreeClassifier

# Simple model, with OrdinalEncoder for the data_channel and weekday categorical
# columns and a DecisionTreeClassifier with default parameter values.
model = make_pipeline(
  OrdinalEncoder(), 
  DecisionTreeClassifier()
)

model.fit(X_train, y_train)


/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


Pipeline(memory=None,
         steps=[('ordinalencoder',
                 OrdinalEncoder(cols=['data_channel', 'weekday'],
                                drop_invariant=False, handle_missing='value',
                                handle_unknown='value',
                                mapping=[{'col': 'data_channel',
                                          'data_type': dtype('O'),
                                          'mapping': culture           1
bus               2
tech              3
world             4
entertainment     5
lifestyle         6
socmed            7
u.s.              8
misc              9
social-good      10
NaN              -2
dtype: int64},
                                         {'col': 'weekday',
                                          'data_type': dt...
                                return_df=True, verbose=0)),
                ('decisiontreeclassifier',
                 DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None,
                        

In [10]:
display(y_train.value_counts(normalize=True))
display(y_val.value_counts(normalize=True))

training_bl = y_train.value_counts(normalize=True).max()
validation_bl = y_val.value_counts(normalize=True).max()

training_acc =  model.score(X_train, y_train)
validation_acc =  model.score(X_val, y_val)


print(f'Training Accuracy:{training_acc:0.4f}/{training_bl:0.4f}')
print(f'Validation Accuracy:{validation_acc:0.4f}/{validation_bl:0.4f}')

0    0.500472
1    0.499528
Name: popularity, dtype: float64

0    0.514281
1    0.485719
Name: popularity, dtype: float64

Training Accuracy:1.0000/0.5005
Validation Accuracy:0.5710/0.5143
